# Machines used

## MacBook Pro Mojave OS

Most tools (except antiSMASH, Nerpa and the ones with webservers) were executed in the MacBook Pro with:

```
Processor: 2.4 GHz Intel Core i5, 8 CPUs.
Memory: 8 GB 2133 MHz LPDDR3.
```

# Installation

## Anaconda

We used the command-line install according to `https://docs.anaconda.com/anaconda/install/mac-os/`.

And the packages:
```
conda install  -c omnia subprocess32
conda install -c bioconda pyfasta
conda install -c conda-forge biopython
```

## antiSMASH v6.0

AntiSMASH can be installed using Anaconda:

`conda install -c bioconda antismash`

or it is possible to use the webservice for a single genomes, like we did for CENA71.

## MASH v2.3

`conda install -c bioconda mash`

## Barrnap 0.9

`conda install -c bioconda -c conda-forge barrnap`

# Downloading genomes

## Inputs

`inputs/wgs_selector_211125.csv` - file with names for cyanobacteria from the NCBI database

`inputs/CENA71.fasta` – genome in fasta format

## Loading packages

In [1]:
import os
import time
from itertools import islice
import pandas as pd
import subprocess
import glob
import string
import re
from pyfasta import Fasta
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

## Downloading genomes

In [5]:
if os.path.exists("./wgs_selector_211125.csv"):
    commands = []
    count = 0
    !cat ./wgs_selector_211125.csv | tr "\t" "~" | cut -d"~" -f1 | sed 1d > ./ids_211125.txt
    with open("./ids_211125.txt") as ids:
        Lines = ids.readlines()
        for line in Lines:
            prefix = line.strip("\n")[:6]
            command = "wget 'ftp://ftp.ncbi.nlm.nih.gov/sra/wgs_aux/%s/%s/%s/%s.1.fsa_nt.gz' -O - | gunzip > ./%s.fasta"%(prefix[0:2],prefix[2:4],prefix,prefix,prefix)
            commands.append(command)
    table1_handle = open('/Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_211125.sh', "w")
    cmd_df = pd.DataFrame(commands)
    cmd_df.to_csv(table1_handle, sep='\t', index=False, header=False)
    table1_handle.close()

```
sh ./download_cyanobacteria_211125.sh
```

In [6]:
!cat /Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_211125.sh | wc -l

    3874


In [37]:
!ls /Volumes/TFL210426/schneider/fasta_genomes/*.fasta | wc -l

    1365


In [29]:
temp_downloaded_list = glob.glob("/Volumes/TFL210426/schneider/fasta_genomes/*.fasta")
downloaded_list = []

for genome_path in temp_downloaded_list:
    downloaded_list.append(os.path.basename(genome_path).split('.')[0])
    
downloaded_list

['AADV02',
 'WJFC01',
 'WGLZ01',
 'WGNO01',
 'WBKL01',
 'WBKM01',
 'WBKN01',
 'WLWJ01',
 'WMIA01',
 'WTFZ01',
 'WTGE01',
 'WTHF01',
 'WTHL01',
 'WTKN01',
 'WVIC01',
 'WVID01',
 'WVIE01',
 'JACAEU01',
 'JACJRZ01',
 'JACYLX01',
 'JACYLY01',
 'JACYMA01',
 'CABUXT',
 'CABXCX',
 'CABUXD01',
 'CACA01',
 'AANP01',
 'AAZV01',
 'ABRS01',
 'ABRV01',
 'ABSE01',
 'ABYK01',
 'ACDW01',
 'ACSK03',
 'AFXD01',
 'AGCR01',
 'AGIK01',
 'AGIZ01',
 'AEPQ01',
 'AESD01',
 'AFEJ01',
 'AFJC01',
 'ACYA01',
 'ACYB01',
 'ADXM01',
 'ALPB01',
 'ALPC01',
 'ALPD01',
 'ALPE01',
 'ALPF01',
 'ALPG01',
 'ALPH01',
 'ALPI01',
 'ALPJ01',
 'ALPK01',
 'ALPL01',
 'ALVI01',
 'ALVJ01',
 'ALVK01',
 'ALVL01',
 'ALVM01',
 'ALVN01',
 'ALVO01',
 'ALVP01',
 'ALVQ01',
 'ALVR01',
 'ALVS01',
 'ALVT01',
 'ALVU02',
 'ALVV02',
 'ALVW01',
 'ALVX01',
 'ALVY01',
 'ALVZ01',
 'ALWA01',
 'ALWB01',
 'ALWC01',
 'ALWD01',
 'AHGV01',
 'AJTX02',
 'AJUB01',
 'AJUS01',
 'AJUT01',
 'AJUU01',
 'AJWF01',
 'AJYB01',
 'AJHB01',
 'AJLJ01',
 'AJLK01',
 'AJLL01'

In [4]:
if os.path.exists("./wgs_selector_211125.csv"):
    commands = []
    count = 0
    !cat ./wgs_selector_211125.csv | tr "\t" "~" | cut -d"~" -f1 | sed 1d > ./ids_211125.txt
    with open("./ids_211125.txt") as ids:
        Lines = ids.readlines()
        for line in Lines:
            prefix = line.strip("\n")[:6]
            if prefix not in downloaded_list:
                command = "wget https://sra-download.ncbi.nlm.nih.gov/traces/wgs01/wgs_aux/%s/%s/%s/%s.1.fsa_nt.gz -O - | gunzip > ./%s.fasta"%(prefix[0:2],prefix[2:4],prefix,prefix,prefix)
                commands.append(command)
    table1_handle = open('/Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_part2_211125.sh', "w")
    cmd_df = pd.DataFrame(commands)
    cmd_df.to_csv(table1_handle, sep='\t', index=False, header=False)
    table1_handle.close()

In [5]:
!cat /Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_part2_211125.sh | wc -l

    2215


```
sh download_cyanobacteria_part2_211125.sh
```

In [58]:
!ls /Volumes/TFL210426/schneider/fasta_genomes/*.fasta | wc -l

    1659


In [30]:
if os.path.exists("./wgs_selector_211125.csv"):
    commands = []
    count = 0
    !cat ./wgs_selector_211125.csv | tr "\t" "~" | cut -d"~" -f1 | sed 1d > ./ids_211125.txt
    with open("./ids_211125.txt") as ids:
        Lines = ids.readlines()
        for line in Lines:
            prefix = line.strip("\n")[:6]
            if prefix not in downloaded_list:
                command = "wget https://sra-download.ncbi.nlm.nih.gov/traces/wgs01/wgs_aux/%s/%s/%s/%s01/%s01.1.fsa_nt.gz -O - | gunzip > ./%s01.fasta"%(prefix[0:2],prefix[2:4],prefix[4:6],prefix,prefix,prefix)
                commands.append(command)
    table1_handle = open('/Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_part3_211221.sh', "w")
    cmd_df = pd.DataFrame(commands)
    cmd_df.to_csv(table1_handle, sep='\t', index=False, header=False)
    table1_handle.close()

In [31]:
!cat /Volumes/TFL210426/schneider/fasta_genomes/download_cyanobacteria_part3_211221.sh | wc -l

    2213


```
sh download_cyanobacteria_part3_211221.sh
```

## Checking for missing genomes to be downloaded manually

In [5]:
temp_downloaded_list = glob.glob("/Volumes/TFL210426/schneider/fasta_genomes/*.fasta")
downloaded_list = []

for genome_path in temp_downloaded_list:
    downloaded_list.append(os.path.basename(genome_path).split('.')[0])
    
downloaded_list

['AADV02',
 'WJFC01',
 'WGLZ01',
 'WGNO01',
 'WBKL01',
 'WBKM01',
 'WBKN01',
 'WLWJ01',
 'WMIA01',
 'WTFZ01',
 'WTGE01',
 'WTHF01',
 'WTHL01',
 'WTKN01',
 'WVIC01',
 'WVID01',
 'WVIE01',
 'JACAEU01',
 'JACJRZ01',
 'JACYLX01',
 'JACYLY01',
 'JACYMA01',
 'CABUCV01',
 'CABXTM01',
 'CABSEX01',
 'CABXSG01',
 'CABXUT01',
 'CABXYB01',
 'CABUXD01',
 'CABYAP01',
 'CABYAW01',
 'CABYEB01',
 'CABYEL01',
 'CABYGI01',
 'CABYIY01',
 'CABYJE01',
 'CABYJI01',
 'CABYKM01',
 'CABYKV01',
 'CABYLC01',
 'CABYLJ01',
 'CABYLR01',
 'CABYMM01',
 'CABYNM01',
 'CABYNZ01',
 'CABYPO01',
 'CABYPR01',
 'CABYQF01',
 'CABYQY01',
 'CABYRK01',
 'CABYRO01',
 'CABYRZ01',
 'CABYSR01',
 'CABYTO01',
 'CABYTR01',
 'CABYUE01',
 'CABYWN01',
 'CABYWR01',
 'CABYWZ01',
 'CABYXI01',
 'CABYYN01',
 'CABYZN01',
 'CABZBE01',
 'CACA01',
 'CABZBP01',
 'CABZCY01',
 'CABZEQ01',
 'CABZFE01',
 'CABZFV01',
 'CABZFW01',
 'CABZHQ01',
 'CABZJH01',
 'CABZJX01',
 'CABZLA01',
 'CABZNT01',
 'CABZOU01',
 'CABZPX01',
 'CABZQQ01',
 'CABZRB01',
 'CABZRX0

In [6]:
if os.path.exists("./wgs_selector_211125.csv"):
    missing_genomes = []
    with open("./ids_211125.txt") as ids:
        Lines = ids.readlines()
        for line in Lines:
            prefix = line.split(",")[0]
            if prefix not in downloaded_list:
                missing_genomes.append(prefix)

In [7]:
for item in missing_genomes:
    print(item)

CABUXT01
CABXCX01
BLAY01
CAJWIC01
CAJWIZ01
CAJWKD01
CAJWNL01
CAJWON01
CAJWSD01
CAJWUF01
CAJWVT01
CAJWWR01
CAJWXK01
CAJWXT01
CAJWZN01
CAJXAP01
CAJXBZ01
CAJXCS01
CAJXFG01
CAJXFQ01
CAJXHD01
CAJXNZ01
CAJXOT01
CAJXPL01
CAJXVP01
CAJYAY01
CAJYBB01
CAJYBI01
CAJYBP01
CAJYBX01
CAJWAH01
CAJWDD01
CAJWDZ01
BPQA01
JAAHBQ01
JAAHFU02
JAAMUW01
JAAMUX01
JACERF01
JAFASY01
JAFAVL01
JADBRW01
JADBRX01
JADBRY01
JADBSA01
JADBSB01
JADBSD01
JADBSE01
JADBSF01
JADBSG01
JADBSH01
JADBSI01
JADBSK01
JADBSL01
JADBSM01
JADBSN01
JADBSO01
JADBSP01
JADBSQ01
JADBSR01
JADBSS01
JADBST01
JADBSU01
JADBSW01
JADBSX01
JADBSY01
JADBSZ01
JADBTB01
JADBTC01
JADBTD01
JADBTE01
JADBTF01
JADBTG01
JADBTH01
JADBTI01
JADBTJ01
JADBTK01
JADBTL01
JADBTO01
JADBTP01
JADBTQ01
JADBTR01
JADBTS01
JADBTT01
JADBTU01
JADBTV01
JADBTW01
JADBTY01
JADBTZ01
JADBUA01
JADBUD01
JADBUE01
JADBUF01
JADBUH01
JADBUI01
JADBUJ01
JADBUL01
JADBUM01
JADBUN01
JADBUO01
JADBUP01
JADBUQ01
JADBUR01
JADBUS01
JADBUT01
JADBUX01
JADBUY01
JADBUZ01
JADBVC01
JADBVE01
JADBVF01
JADBV

In [13]:
for item in missing_genomes:
    if "NZ_" in item:
        new_item = item[3:]
        if new_item in missing_genomes:
            print(new_item)

PGOO01
PGOP01
PGOQ01
PGOR01
PGOS01
PGOT01
PGOU01
PGOV01
PGOW01
PGOY01
PGWP01
PHNM01
BPQA01
BLAY01
VTKY01
VTKZ01
VTLC01
RPHB01


---

In [11]:
temp_downloaded_list = glob.glob("/Volumes/TFL210426/schneider/fasta_genomes/*.fasta")
downloaded_list = []

for genome_path in temp_downloaded_list:
    downloaded_list.append(os.path.basename(genome_path).split('.')[0])
    
downloaded_list

['AADV02',
 'WJFC01',
 'WGLZ01',
 'WGNO01',
 'WBKL01',
 'WBKM01',
 'WBKN01',
 'WLWJ01',
 'WMIA01',
 'WTFZ01',
 'WTGE01',
 'WTHF01',
 'WTHL01',
 'WTKN01',
 'WVIC01',
 'WVID01',
 'WVIE01',
 'JACAEU01',
 'JACJRZ01',
 'JACYLX01',
 'JACYLY01',
 'JACYMA01',
 'CABUCV01',
 'CABXTM01',
 'CABSEX01',
 'CABXSG01',
 'CABXUT01',
 'CABXYB01',
 'CABUXD01',
 'CABYAP01',
 'CABYAW01',
 'CABYEB01',
 'CABYEL01',
 'CABYGI01',
 'CABYIY01',
 'CABYJE01',
 'CABYJI01',
 'CABYKM01',
 'CABYKV01',
 'CABYLC01',
 'CABYLJ01',
 'CABYLR01',
 'CABYMM01',
 'CABYNM01',
 'CABYNZ01',
 'CABYPO01',
 'CABYPR01',
 'CABYQF01',
 'CABYQY01',
 'CABYRK01',
 'CABYRO01',
 'CABYRZ01',
 'CABYSR01',
 'CABYTO01',
 'CABYTR01',
 'CABYUE01',
 'CABYWN01',
 'CABYWR01',
 'CABYWZ01',
 'CABYXI01',
 'CABYYN01',
 'CABYZN01',
 'CABZBE01',
 'CACA01',
 'CABZBP01',
 'CABZCY01',
 'CABZEQ01',
 'CABZFE01',
 'CABZFV01',
 'CABZFW01',
 'CABZHQ01',
 'CABZJH01',
 'CABZJX01',
 'CABZLA01',
 'CABZNT01',
 'CABZOU01',
 'CABZPX01',
 'CABZQQ01',
 'CABZRB01',
 'CABZRX0

In [24]:
if os.path.exists("./wgs_selector_211125.csv"):
    missing_genomes = []
    with open("./ids_211125.txt") as ids:
        Lines = ids.readlines()
        for line in Lines:
            if 'NZ_' in line:
                prefix = line.split(",")[0]
                prefix = prefix[3:]
            else:
                prefix = line.split(",")[0]
            if prefix not in downloaded_list:
                missing_genomes.append(prefix)

In [25]:
missing_genomes

[]

## Running MASH

In [9]:
!ls /Volumes/TFL210426/schneider/fasta_genomes/*.fasta | wc -l

    3400


https://github.com/marbl/Mash/releases

In [10]:
start_time = time.time()

if os.path.exists("./mash-OSX64-v2.3/ncbi_genomes_3.msh"):
    print("Sketch already exists")
else:
    subprocess.call("./mash-OSX64-v2.3/mash sketch -o ./mash-OSX64-v2.3/ncbi_genomes_3 /Volumes/TFL210426/schneider/fasta_genomes/*.fasta",shell=True)
    
print('\n' + "--- %s seconds ---" %(time.time()-start_time))


--- 1037.9838089942932 seconds ---


In [14]:
def run_mash_df(fasta_input,sketch):
    start_time = time.time()
    cmd = "./mash-OSX64-v2.3/mash dist %s %s > ./mash-OSX64-v2.3/mash_df_3.txt"%(sketch,fasta_input)
    subprocess.call(cmd,shell=True)
    print('\n' + "--- %s seconds ---" %(time.time()-start_time))
        
run_mash_df("./CENA71.fasta","./mash-OSX64-v2.3/ncbi_genomes_3.msh")


--- 0.7175679206848145 seconds ---


In [15]:
mash_df = pd.read_csv("./mash-OSX64-v2.3/mash_df_3.txt",sep="\t",names=["reference","query","distance","pvalue","matching-hashes"])

mash_df = mash_df.sort_values(by=['distance'])

mash_df[:20]

,reference,query,distance,pvalue,matching-hashes
29,/Volumes/TFL210426/schneider/fasta_genomes/AJL...,./CENA71.fasta,0.074070,0.000000e+00,118/1000
62,/Volumes/TFL210426/schneider/fasta_genomes/ALV...,./CENA71.fasta,0.079484,0.000000e+00,104/1000
2197,/Volumes/TFL210426/schneider/fasta_genomes/LIR...,./CENA71.fasta,0.079901,0.000000e+00,103/1000
67,/Volumes/TFL210426/schneider/fasta_genomes/ALV...,./CENA71.fasta,0.079901,0.000000e+00,103/1000
2268,/Volumes/TFL210426/schneider/fasta_genomes/MNP...,./CENA71.fasta,0.081614,0.000000e+00,99/1000
2321,/Volumes/TFL210426/schneider/fasta_genomes/NAP...,./CENA71.fasta,0.082054,0.000000e+00,98/1000
3086,/Volumes/TFL210426/schneider/fasta_genomes/QLK...,./CENA71.fasta,0.083404,0.000000e+00,95/1000
1959,/Volumes/TFL210426/schneider/fasta_genomes/JAH...,./CENA71.fasta,0.086249,0.000000e+00,89/1000
30,/Volumes/TFL210426/schneider/fasta_genomes/AJL...,./CENA71.fasta,0.126974,9.205260e-154,36/1000
2404,/Volumes/TFL210426/schneider/fasta_genomes/NRQ...,./CENA71.fasta,0.126974,6.184730e-154,36/1000


## Polishing the dataframe

In [26]:
from Bio import SeqIO

def split_string_at(s, c, n):
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])

def extract_ncbi_taxa(inputs_folder):
    input_list = glob.glob("%s*.fasta"%inputs_folder)
    taxa = {}
    for item in input_list:
        records = list(SeqIO.parse(item, "fasta"))
        strain = os.path.splitext(os.path.basename(item))[0]
        if records:
            if "TPA_asm" in str(records[0]):
                genera = split_string_at(genera,":",1)[0]
            else:
                genera = split_string_at(records[0].description," ",1)[1]
            if "Candidatus" in str(genera):
                taxa[strain] = str(genera).partition(' ')[2].partition(' ')[0]
            else:
                taxa[strain] = str(genera).partition(' ')[0].lstrip("[").rstrip("]")
    return taxa
            
taxa_dict = extract_ncbi_taxa("/Volumes/TFL210426/schneider/fasta_genomes/")

In [27]:
taxa_dict

{'AADV02': 'Crocosphaera',
 'WJFC01': 'Scytonema',
 'WGLZ01': 'Cyanobacteria',
 'WGNO01': 'Nostoc',
 'WBKL01': 'Nostoc',
 'WBKM01': 'Nostoc',
 'WBKN01': 'Nostoc',
 'WLWJ01': 'Synechococcus',
 'WMIA01': 'Cyanobacterium',
 'WTFZ01': 'Synechococcus',
 'WTGE01': 'Synechococcus',
 'WTHF01': 'Synechococcus',
 'WTHL01': 'Synechococcus',
 'WTKN01': 'MAG:',
 'WVIC01': 'Synechococcales',
 'WVID01': 'Nostoc',
 'WVIE01': 'Myxacorys',
 'JACAEU01': 'Fischerella',
 'JACJRZ01': 'Fischerella',
 'JACYLX01': 'Fischerella',
 'JACYLY01': 'Fischerella',
 'JACYMA01': 'Fischerella',
 'CABUCV01': 'Fischerella',
 'CABXTM01': 'uncultured',
 'CABSEX01': 'uncultured',
 'CABXSG01': 'uncultured',
 'CABXUT01': 'uncultured',
 'CABXYB01': 'uncultured',
 'CABUXD01': 'uncultured',
 'CABYAP01': 'uncultured',
 'CABYAW01': 'uncultured',
 'CABYEB01': 'uncultured',
 'CABYEL01': 'uncultured',
 'CABYGI01': 'uncultured',
 'CABYIY01': 'uncultured',
 'CABYJE01': 'uncultured',
 'CABYJI01': 'uncultured',
 'CABYKM01': 'uncultured',
 

In [28]:
count = 0

for key in taxa_dict:
    if taxa_dict[key] == 'Fischerella':
        count += 1
        
count

49

In [29]:
percent_col = []
new_name = []

for i,r in mash_df.iterrows():
    percent_col.append(int(r['matching-hashes'].split('/')[0])/10)
    ncbi_code = os.path.basename(r['reference'].split('.')[0])
    new_name.append(ncbi_code+'_'+taxa_dict[ncbi_code])
    
    
mash_df['percentage'] = percent_col
mash_df['reference'] = new_name

mash_df[:20]

,reference,query,distance,pvalue,matching-hashes,percentage
29,AJLJ01_Fischerella,./CENA71.fasta,0.074070,0.000000e+00,118/1000,11.8
62,ALVS01_Fischerella,./CENA71.fasta,0.079484,0.000000e+00,104/1000,10.4
2197,LIRN01_Hapalosiphon,./CENA71.fasta,0.079901,0.000000e+00,103/1000,10.3
67,ALVX01_Fischerella,./CENA71.fasta,0.079901,0.000000e+00,103/1000,10.3
2268,MNPM02_Mastigocladus,./CENA71.fasta,0.081614,0.000000e+00,99/1000,9.9
2321,NAPS01_Westiellopsis,./CENA71.fasta,0.082054,0.000000e+00,98/1000,9.8
3086,QLKN01_Hapalosiphonaceae,./CENA71.fasta,0.083404,0.000000e+00,95/1000,9.5
1959,JAHHHW01_MAG:,./CENA71.fasta,0.086249,0.000000e+00,89/1000,8.9
30,AJLK01_Fischerella,./CENA71.fasta,0.126974,9.205260e-154,36/1000,3.6
2404,NRQW01_Fischerella,./CENA71.fasta,0.126974,6.184730e-154,36/1000,3.6


In [30]:
mash_df.to_csv("./mash_df_fischerella-220111.csv",sep='\t')

In [33]:
!mkdir top_100

In [35]:
for genome in mash_df['reference'][:100]:
    subprocess.call('cp /Volumes/TFL210426/schneider/fasta_genomes/%s.fasta ./top_100/'%genome.split('_')[0],shell=True)

## Extracting 16S rRNA

```
barrnap CENA71.fasta > barrnap_rRNAs.txt
```

In [106]:
def extract_nucleotides(fastafile,contig_name,start,end):
    if os.path.exists(fastafile):
        f = Fasta(fastafile)
        nucleotides = f[str(contig_name)][start:end]
    else:
        raise ValueError("Fasta file not found")
    return nucleotides

def fetch_barrnap_rRNA_genes(barrnap_df,rRNA,output_file):
    rRNA_options = ['5S','16S','23S']
    if rRNA not in rRNA_options:
        raise ValueError("Select '5S', '16S' or '23S' as the rRNA to be fetched")
    else:
        records = []
        for i,r in barrnap_df.iterrows():
            m = re.search(r'%s'%rRNA,barrnap_df.i.loc[i])
            if m:
                fastafile = "CENA71.fasta"
                if os.path.exists(fastafile):
                    nucleotides = extract_nucleotides(fastafile,barrnap_df.a.loc[i],
                                                      int(barrnap_df.d.loc[i]),int(barrnap_df.e.loc[i]))
                    seq = SeqRecord(Seq(nucleotides,generic_dna), id = "%s"%barrnap_df.a.loc[i], description="%s_gene"%rRNA)
                    records.append(seq)
                else:
                    raise ValueError("Fasta file %s not found"%fastafile)
    handle = open("%s"%output_file, "w")
    SeqIO.write(records,handle,"fasta")
    handle.close
    return records

def extract_rRNA_genes(df,barrnap_input,output_file,rRNA):
    if os.path.exists(output_file):
        print("%s genes fasta output already exists"%rRNA)
    else:
        start_time = time.time()
        records = fetch_barrnap_rRNA_genes(df,rRNA,output_file)
        count = len(records)
        print("%s %s genes extracted! Fasta files saved as %s and %s.ren.fasta."%(count,rRNA,output_file,
                                                                                  os.path.splitext(output_file)[0]))
        print('\n' + "--- %s seconds ---" %(time.time()-start_time))
        
barrnap_file = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/barrnap_rRNAs.txt"
barrnap_df = pd.read_csv(barrnap_file,names=list(string.ascii_lowercase)[0:9],sep='\t').dropna()

extract_rRNA_genes(barrnap_df,barrnap_file,"/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/barrnap_16S.fasta","16S")

1 16S genes extracted! Fasta files saved as /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/barrnap_16S.fasta and /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/barrnap_16S.ren.fasta.

--- 0.008651018142700195 seconds ---
